# OpenAI Response API with Structured Output

This example demonstrates how to use Structured Output with OpenAI response API client.

In [ ]:
import json
import os

from pydantic import BaseModel

import autogen


# ---------------------------------------------------------------------
# 1. Define the response format (a Pydantic model)
# ---------------------------------------------------------------------
class QA(BaseModel):
    question: str
    answer: str
    reasoning: str


# ---------------------------------------------------------------------
# 2. Build an llm_config that opts-in to the Responses endpoint
#    and attaches the structured-output model
# ---------------------------------------------------------------------
llm_config = {
    "config_list": [
        {
            "api_type": "responses",  # <─ use /responses
            "model": "gpt-4o",  # any supported model
            "api_key": os.getenv("OPENAI_API_KEY"),
            "response_format": QA,  # <─ structured output!
        }
    ]
}

# ---------------------------------------------------------------------
# 3. Create two simple chat actors
# ---------------------------------------------------------------------
user = autogen.UserProxyAgent(
    name="User",
    system_message="Human admin",
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="StructuredBot",
    llm_config=llm_config,
    system_message=(
        "You are a Q&A bot. Always return a JSON object that matches the QA schema: {question, answer, reasoning}"
    ),
)

# ---------------------------------------------------------------------
# 4. Start the conversation
# ---------------------------------------------------------------------
result = user.initiate_chat(
    assistant,
    message="What causes seasons on Earth?",
    max_turns=1,
    summary_method="last_msg",
)

In [ ]:
print("RAW LLM OUTPUT:", result.summary)  # already parsed!
print()
print("Pretty-printed ➜")
obj = json.loads(result.summary)  # or result.summary if you added .format()
print(json.dumps(obj, indent=2))

# Expected output:
# RAW LLM OUTPUT: {"question":"What causes seasons on Earth?","answer":"The tilt of Earth's rotational axis relative to its orbital plane.","reasoning":"Earth’s axis is tilted about 23.5°..."}